# F1 Racing Bets Development
> Development Blog/Guide to F1 Betting

- title: F1 Racing Bets Development
- badges: true
- toc: true
- comments: true
- categories: [pbl]

### Overview
Over the course of the project our group has been doing, one key feature we decided to add was betting. In F1 Racing, one key part of the sport for viewers is betting on teams. Thus, for the user side, we decided to add the betting feature to allow users to bet for races using a virtual currency we've coined as "F1 Coins" (pun unintended). Let's take a look at the front end first!

#### Frontend
![login](https://user-images.githubusercontent.com/75715248/221756801-d249e315-9bda-445b-9ff0-082a3728b853.png)

Of course, the first thing to do would be to log into a user (in this case someone with the role of audience). The most important thing we want from our log in is the JWT cookie that comes with logging in which importantly stores the users id. Through the user id, we can now proceed to the perfectly legal and fun part of our site, betting. 

> Note: without logging in, the betting page will automatically redirect to the log in page prompting it in order to use the betting page. 

![bet page](https://user-images.githubusercontent.com/75715248/221758430-14014828-8c17-469b-9665-772f8ecb0253.png)

Now onto bets! Although not as clean as I would like (and besides the scuffed html), the bets page has all of the functionality necessary to demonstrate the backend that was developed for bets. Besides the scuffed text inputs and messy table, the slideshow and increamental f1 coin inputs are the key front end portions (thanks to Nathan for his work on a lot of the front end html elements and CSS).

> If the html were to be better developed, the gray box would store dynamic listings of races that would allow the user to select what race they would like to bet for. For now, the data necessary for the API call RequestBody is handled in simple text inputs. 

Starting with the top right, we have the f1 coins that each user has in their possession (starts at 100). This is accomplished by taking the user id stored in local storage (props to Aadit) which can then be passed through as a parameter into the API request retrieving the f1coin amount of any user. 

Moving to the actual betting part, in the red area of the page, we have a slide show that translates into specific teams as an index for the slide show. That index is then translated into its corresponding team when the submit button is clicked (currently, custom teams are not supported given the exclusivity of F1 Racing). Below that is the numeber input for f1 coins. Moving to below the main betting interface is the temporary race season and race name text inputs which are meant to be passed for the required request body fields. Given a functional frontend, the race season and name would be handled by the race they select. Once the submit button is clicked, all of the above values are passed into a body that is passed to the api fetch which creates a bet. If a field is missing or is wrongly inputted, an alert for a failure to make the bet will pop up.

Finally, below is a list of the bets and url options to update the bets and delete them. The update link will send the user to a new page in which they must pass the bet id and the changes to team and f1 coins they are making (could be better esp. since the link does not actually store the bet id as it should). The delete will delete the bet from the database.

#### Backend
So how does the bet creation, updating, deleting, and other related APIs, JPAs, and methods work? Let's start with the bets themselves. For our entire database, much of the POJO's stored are weaved in a complex web of relationships which applies most to the Bet POJO we created. 

![bet object](https://user-images.githubusercontent.com/75715248/221762181-ff29f276-40c7-4449-a07a-29d1d3fb02f5.png)

As seen above, beyond the standard fields stored are three ManyToOne relationships with join columns. The CascadeType.MERGE for each of the relationships serves to enable the changes to one side of the relationship to be mirrored in the other POJOs in the relationship. The resulting Bet object is then stored in a BetJPARepository which has the following functions used to query the database and return them.

![smth](https://user-images.githubusercontent.com/75715248/221762951-6fbf851a-15c3-408e-9903-04d2fcc750e5.png)

I'll skim over the other objects but they are also all equally complex using JPA repositories to store them.

For the api controller, this is all handled under user given it is an action done by the user. For the make bet, most of it is standard creating a pojo and saving it to the JPA repo. All missing inputs are checked and returns a bad request if bad. If the time the bet is returned is after the race date, it also returns an error. This also changes the f1coin amounts of users. The make bets uses jpa repository for users, teams, and races (this is a table created on initialization calling the API) An override exists for testing in PostMan. Update and delete are pretty standard as well with the update to an existing bet and delete to delete an existing bet. A processBets() method exists as a way to process a bet by a date to check who the winner is (this is used for testing in postman).

![ewdsafasdf](https://user-images.githubusercontent.com/75715248/221766063-bc662580-f0a9-49a1-8669-ffde17efcc04.png)

Finally, the last part to our backend for bets is the periodic methods that are ran to find race winners on that day and to process bets on that given day. This is done through SpringBoot's Scheduler annotations which allows the periodic running of methods. In this case, we have it run every 20 minutes. For race results, this is all done using ergast's api to get winners for a given day which then stores it into the race object through the race jpa repo. For processing bets, each bet given a specific race has the team that the bet was for compared to the winner of the race and updated accordingly ultimately leaving the bet inactive after the bet has been processed.

### End
That's all for bets. Although there were a fair amount of things in regards to bets that I could have covered, these are the key features and overview of the entire feature itself and I hope it helps in understanding the development of this feature.
